In [1]:
import cv2
import numpy as np
from object_detection import ObjectDetection

In [17]:
YOLOS = [ ["dnn_model/yolo-fastest-1.1.weights", "dnn_model/yolo-fastest-1.1.cfg", "dnn_model/coco.names.txt"],
        
        ["dnn_model/yolo-fastest-1.1-xl.weights", "dnn_model/yolo-fastest-1.1-xl.cfg", "dnn_model/coco.names.txt"],
        ["dnn_model/yolov4.weights", "dnn_model/yolov4.cfg", "dnn_model/classes.txt"]]

i = 2
YOLO = ObjectDetection(weights_path = YOLOS[i][0], 
                       cfg_path = YOLOS[i][1],
                      class_path = YOLOS[i][2])

Loading Object Detection
Running opencv dnn with YOLOv4


In [18]:
def loadClassesNames():
        with open('dnn_model/classes.txt') as f:
            Objects = [line.replace('\n','') for line in f.readlines()]
        f.close()
        return Objects

In [19]:
class Object():
    All = {}
    LastID = 0
    Threshold = 50
    MaxFramesWithOutNoMove = 20
    TemporalStep = 1
    ClassesNames = loadClassesNames()
    
    @classmethod
    def create(cls, name, PosIni):
        """
        if ID in cls.All.keys():
            raise IndexError(f"No puede existir dos ID's iguales ID: {ID}")
        
        if len(list(cls.All.keys())) >1 and ID != np.max(list(cls.All.keys())) + 1:
            txtErr = f"Los ID deben de ser consecutivos ID: {ID}"
            txtErr += f"\nSiguiente ID: {np.max(list(cls.All.keys())) + 1}"
            raise IndexError(txtErr)
        """
        object_ = Object(name, PosIni)
        cls.All[Object.LastID] = object_
        return object_
    
    @classmethod
    def NewFrame(cls, Coords, Classes):
        
        for coor, class_ in zip(Coords, Classes):
            min_dis = 100
            min_dis_ID = None
            for ID in Object.All.keys():
                d = Object.All[ID].getDistance(coor)
                if d < Object.Threshold and d < min_dis and Object.All[ID].Life == True:
                    min_dis = d
                    min_dis_ID = ID
            # Si está más cerca de un objeto, se actualiza su posición
            if min_dis_ID != None and Object.All[min_dis_ID].name == Object.ClassesNames[class_]:
                Object.All[min_dis_ID].ActPosition(coor)
                Object.All[min_dis_ID].LastTemporalStep = Object.TemporalStep
            #Si no, es un objeto nuevo
            else:
                Object.create(class_, coor)
                Object.All[Object.LastID].LastTemporalStep = Object.TemporalStep
        
        Object.TemporalStep += 1
        Object.checkDeath()
        
    @classmethod
    def ShowAll(cls, onlyALives = False):
        for k in Object.All.keys():
            if onlyALives:
                if Object.All[k].Life == False:
                    continue
                    
            print(" ================================== ")
            print(f"ID: {Object.All[k].ID}")
            print(f"Tipo: {Object.All[k].name}")
            print(f"Path: {Object.All[k].path}")
            print(f"Pos: {Object.All[k].getPosition()}")
            print(" ================================== ")
            print()
            
    @classmethod
    def checkDeath(cls):
        
        for ID in Object.All.keys():
            Diff = Object.TemporalStep - Object.All[ID].LastTemporalStep
            if Diff > Object.MaxFramesWithOutNoMove:
                Object.All[ID].Life = False
                print(f"Murió ID = {ID}")
                
    @classmethod
    def PutID(cls, frame):
        onlyALives = True
        for k in Object.All.keys():
            if onlyALives:
                if Object.All[k].Life == False:
                    continue
            
            x, y = Object.All[k].getPosition()
            cv2.putText(frame, str(Object.All[k].ID), (x ,y-7), 0,1, (0,0,255), 2)
        
    
    def __init__(self, name, PosIni):
        self.name = Object.ClassesNames[name]
        self.PosIni = PosIni
        self.Life = True
        self.LastTemporalStep = 1
        self.ID = Object.LastID
        self.path = [PosIni]
        Object.LastID += 1
        
    def ActPosition(self, ActPos):
        self.path.append(ActPos)
    
    def getPosition(self):
        return self.path[-1]
    
    def checkMove(self):
        """
        Si un objeto permanece inmovil por más de n frames,
        se considera como muerto
        """
        pass
        #if len(self.path) > 10:
            
    
    def getDistance(self, Pos):
        return np.sqrt((Pos[0] - self.getPosition()[0])**2 +\
                       (Pos[1] - self.getPosition()[1])**2)

In [20]:
cap = cv2.VideoCapture('videos/short.mp4')
nFrame = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    key = cv2.waitKey(1)
    
    
    roi = frame[500:, :]
    #"""
    Object.PutID(roi)
    if nFrame% 1 == 0:
        (class_ids, scores, boxes) = YOLO.detect(roi)
        for box, class_id in zip(boxes, class_ids):
            (x, y, w, h) = box
            cv2.rectangle(roi, (x,y), (x + w, y + h), (255,0,0),2)
            cx = int((x + x + w)/2)
            cy = int((y + y + h)/2)
            cv2.circle(roi, (cx,cy), 5, (0,0,255), -1 )


            Object.NewFrame([[cx,cy]], [class_id])

            

            Object.ShowAll(onlyALives = True)
    #"""
    
    cv2.imshow("Frame", frame)
    cv2.imshow("Roi", roi)
    if key == 27:
        break  
    
    nFrame += 1
        
cap.release()
cv2.destroyAllWindows()

ID: 0
Tipo: car
Path: [[477, 40]]
Pos: [477, 40]

ID: 0
Tipo: car
Path: [[477, 40], [477, 40]]
Pos: [477, 40]

ID: 0
Tipo: car
Path: [[477, 40], [477, 40], [477, 40]]
Pos: [477, 40]

ID: 0
Tipo: car
Path: [[477, 40], [477, 40], [477, 40], [477, 40]]
Pos: [477, 40]

ID: 0
Tipo: car
Path: [[477, 40], [477, 40], [477, 40], [477, 40], [477, 40]]
Pos: [477, 40]

ID: 0
Tipo: car
Path: [[477, 40], [477, 40], [477, 40], [477, 40], [477, 40], [477, 40]]
Pos: [477, 40]

ID: 0
Tipo: car
Path: [[477, 40], [477, 40], [477, 40], [477, 40], [477, 40], [477, 40], [477, 40]]
Pos: [477, 40]



In [170]:
Object.NewFrame([[20.4,10.4]], [3])

In [171]:
Object.ShowAll(onlyALives=True)

ID: 0
Tipo: motorbike
Path: [[20.1, 10.1], [20.1, 10.1], [20.1, 10.1], [20.2, 10.2], [20.3, 10.3], [20.4, 10.4]]
Pos: [20.4, 10.4]



In [24]:
f.close()

In [67]:
np.array(Object.All[1].path[-5:])

array([[478, 540],
       [478, 539],
       [477, 540],
       [477, 539],
       [477, 539]])

In [180]:
x,y = Object.All[1].getPosition()

In [181]:
   x,y

(477, 539)

In [ ]:
# Para una nueva versión, eliminarlos en lugar de matarlos en código
# del Object.All[ID]
# Pero antes revisar a donde se han ido
# detección cada n frames